In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader
from PIL import Image

import numpy as np
import matplotlib.pyplot as plt

In [2]:
class DeepDream(nn.Module):
    def __init__(self, model, layer_idx):
        super(DeepDream, self).__init__()
        self.features = self.get_required_layers(model, layer_idx)

    def forward(self, x):
        return self.features(x)

    def get_required_layers(self, model, layer_idx):
        if isinstance(model, nn.Sequential):
            return nn.Sequential(*list(model.children())[:layer_idx+1])
        elif isinstance(model, nn.Module):
            return model  # You might need to adjust this depending on the structure of your model
        else:
            raise ValueError("Unsupported model type")

In [3]:
def deep_dream(image_tensor, model, layer_idx, iterations, lr, octave_scale, output_path):
    # Convert image tensor to nn.Parameter
    img = nn.Parameter(image_tensor.to(device))

    # Define the deep dream model
    dream_model = DeepDream(model, layer_idx).to(device)
    
    # Define the optimizer
    optimizer = optim.Adam([img], lr=lr)
    
    # DeepDream iterations
    for i in range(iterations):
        optimizer.zero_grad()
        features = dream_model(img)
        loss = features.norm()
        loss.backward()
        optimizer.step()
    
        # Apply the octave scaling
        img.data = img.data + octave_scale * img.grad.data
    
        # Zero the gradient
        img.grad.data.zero_()
    
        # Clip the image values to be in the valid range
        img.data = torch.clamp(img.data, 0, 1)
    
    # Save the final deep dream image
    result = transforms.ToPILImage()(img.squeeze(0).cpu())
    result.save(output_path)

In [4]:
# Set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Specify the layer index for deep dream (you can experiment with different layers)
layer_index = 10

iterations = 20
learning_rate = 0.01
octave_scale = 1.4

In [5]:
model = models.inception_v3(pretrained=True).eval().to(device)

/home/user/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/user/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [6]:
train_transform = transforms.Compose([
    transforms.Resize((299, 299)),  # Resize images to the same dimensions as expected by InceptionV3
    transforms.ToTensor()
])

test_transform = transforms.Compose([
    transforms.Resize((299, 299)),  # Resize images to the same dimensions as expected by InceptionV3
    transforms.ToTensor()
])

In [7]:
train_dataset = datasets.ImageFolder(root='../data/train', transform=train_transform)
test_dataset = datasets.ImageFolder(root='../data/validation', transform=test_transform)

In [8]:
for i in range(2000):
    img, label = train_dataset[i]
    output_image_path = f'deepdream_pt_train_pic{i}.jpg'
    deep_dream(img.reshape(1, img.shape[0], img.shape[1], img.shape[2]), model, layer_index, iterations, learning_rate, octave_scale, output_image_path)

In [9]:
for i in range(1000):
    img, label = test_dataset[i]
    output_image_path = f'deepdream2_pt_test_pic{i}.jpg'
    deep_dream(img.reshape(1, img.shape[0], img.shape[1], img.shape[2]), model, layer_index, iterations, learning_rate, octave_scale, output_image_path)